## Project III: Police Brutality

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, logistic
import seaborn as sns
sns.set_theme()

# User-written modules (assuming they are saved as estimation.py, LinearModel.py, probit_ante.py, and logit_ante.py)
import estimation as est
import LinearModel as lm
import probit_ante as probit
import logit_ante as logit

from scipy.stats import t

%load_ext autoreload
%autoreload 2

## Load data and recode categorical data for income and population

'sincome' has 3 categories: "Under $20,000", "$20,000-$50,000" and "Over $50,000"

I define them as low, medium and high

'spop' has 4 categories: "Under 100,000", "100,000-499,000", "500,000-999,999" and "Over 1 million"

I define them as small, medium, large and huge


In [2]:
# Load the data
data = pd.read_csv('ppcs_cc.csv')

data['sincome_low'] = (data['sincome'] == 1).astype(int)
data['sincome_medium'] = (data['sincome'] == 2).astype(int)
data['sincome_high'] = (data['sincome'] == 3).astype(int)

data['spop_small'] = (data['spop'] == 1).astype(int)
data['spop_medium'] = (data['spop'] == 2).astype(int)
data['spop_large'] = (data['spop'] == 3).astype(int)
data['spop_huge'] = (data['spop'] == 4).astype(int)

data['inctype_street'] = (data['inctype_lin'] == 1).astype(int)
data['inctype_traffic'] = (data['inctype_lin'] == 2).astype(int)

data['sage_scaled'] = data['sage'] / 10

## Construct dataset

I only include data without multicollinarity problems.

NOTE: Only 3 out of all the black people in the dataset have experienced police violence

In [3]:
varlist = ['smale', 'sempl', 'sincome_medium', 'sincome_high', 'spop_medium', \
           'spop_large', 'spop_huge', 'inctype_traffic', 'daytime', 'sbehavior', 'sage_scaled']

y_lab = ['anyuseofforce_coded']
x_lab_c = ['sblack', 'shisp', 'sother'] + varlist + ['const']
continoues_varlist_c = [False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]

x_lab_nc = ['sblack', 'shisp', 'sother', 'const']
continoues_varlist_nc = [False, False, False, False]

# Prepare data for models without controls
y = data['anyuseofforce_coded'].values
N = len(y)
x_nc = data[['sblack', 'shisp', 'sother']].values
x_nc = np.hstack([x_nc, np.ones((N, 1))])  # Add constant term

# Prepare data for models with controls
x_c = data[['sblack', 'shisp', 'sother'] + varlist].values
x_c = np.hstack([x_c, np.ones((N, 1))])  # Add constant term 

In [4]:
pd.set_option('display.precision', 2)

varlist_table = ['swhite', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome_low', 'sincome_medium', 'sincome_high', 'spop_small', 'spop_medium', \
           'spop_large', 'spop_huge', 'inctype_traffic', 'daytime', 'sbehavior'] \
            + ['omajwhite', 'omajblack', 'omajhisp', 'omajother']

# Placeholder for the results
results = []

# Loop through each variable in the variable list
for var in varlist_table:
    not_used_force = data[data['anyuseofforce_coded'] == 0]

    count_not_force = int(not_used_force[var].sum())  # Sum counts for 'not force'
    mean_not_force = not_used_force[var].mean()  # Mean for 'not force'

    used_force = data[data['anyuseofforce_coded'] == 1]

    count_force = int(used_force[var].sum())  # Sum counts for 'force used'
    mean_force = used_force[var].mean()  # Mean for 'force used'

    # Append results to the list
    results.append({
        'Variable': var,
        'Count (no force) ': count_not_force,
        'Share (no force)': mean_not_force,
        'Count (force)': count_force,
        'Share (force)': mean_force
    })

# Convert the results into a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
results_df


,Variable,Count (no force),Share (no force),Count (force),Share (force)
0,swhite,2799,0.74,9,0.47
1,sblack,417,0.11,3,0.16
2,shisp,380,0.10,6,0.32
3,sother,184,0.05,1,0.05
4,smale,1997,0.53,15,0.79
5,sempl,2633,0.70,9,0.47
6,sincome_low,1099,0.29,6,0.32
7,sincome_medium,957,0.25,6,0.32
8,sincome_high,1724,0.46,7,0.37
9,spop_small,2913,0.77,8,0.42


In [5]:
latex_table = results_df.to_latex(
    index=False,  # Do not include the DataFrame index in the LaTeX table
    float_format="%.2f",  # Format floating-point numbers to 2 decimal places
    caption="Summary statistics for variables based on use of force",
    label="tab:use_of_force_summary"
)

# Print the LaTeX table
print(latex_table)

\begin{table}
\caption{Summary statistics for variables based on use of force}
\label{tab:use_of_force_summary}
\begin{tabular}{lrrrr}
\toprule
Variable & Count (no force)  & Share (no force) & Count (force) & Share (force) \\
\midrule
swhite & 2799 & 0.74 & 9 & 0.47 \\
sblack & 417 & 0.11 & 3 & 0.16 \\
shisp & 380 & 0.10 & 6 & 0.32 \\
sother & 184 & 0.05 & 1 & 0.05 \\
smale & 1997 & 0.53 & 15 & 0.79 \\
sempl & 2633 & 0.70 & 9 & 0.47 \\
sincome_low & 1099 & 0.29 & 6 & 0.32 \\
sincome_medium & 957 & 0.25 & 6 & 0.32 \\
sincome_high & 1724 & 0.46 & 7 & 0.37 \\
spop_small & 2913 & 0.77 & 8 & 0.42 \\
spop_medium & 545 & 0.14 & 7 & 0.37 \\
spop_large & 151 & 0.04 & 1 & 0.05 \\
spop_huge & 171 & 0.05 & 3 & 0.16 \\
inctype_traffic & 3628 & 0.96 & 13 & 0.68 \\
daytime & 2523 & 0.67 & 9 & 0.47 \\
sbehavior & 237 & 0.06 & 10 & 0.53 \\
omajwhite & 3415 & 0.90 & 18 & 0.95 \\
omajblack & 231 & 0.06 & 0 & 0.00 \\
omajhisp & 90 & 0.02 & 1 & 0.05 \\
omajother & 44 & 0.01 & 0 & 0.00 \\
\bottomrule
\end{

In [6]:
ols = lm.estimate(y, x_nc)
ols_tab = lm.print_table((y_lab, x_lab_nc), ols, title='LPM results')
ols_tab

LPM results
Dependent variable: ['anyuseofforce_coded']

R2 = 0.003
sigma2 = 0.005


,b_hat,se,t
sblack,3.90e-03,3.70e-03,1.07
shisp,1.23e-02,3.80e-03,3.23
sother,2.20e-03,5.30e-03,0.41
const,3.20e-03,1.30e-03,2.41


## Main function

Define the main function to calculate both probit and logit models

In [7]:
def probit_logit_model(x_vars, x_lab, continoues_varlist, options, method='BFGS', constant=1, average_effect=True):

    theta0 = probit.starting_values(y, x_vars)
    probit_results = est.estimate(probit.q, theta0, y, x_vars, options=options, method=method)

    probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')

    theta0 = logit.starting_values(y, x_vars)
    logit_results = est.estimate(logit.q, theta0, y, x_vars, options=options, method=method)
    logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')

    b_pr = probit_tab.theta.values
    b_lg = logit_tab.theta.values

    results = {}

    for k in range(x_vars.shape[1]-constant): 

        me_variable_pr_list = []
        me_variable_lg_list = []

        grad_pr_list = []
        grad_lg_list = []

        if average_effect:
            x_vars_temp_ = np.mean(x_vars, axis=0).reshape(1, -1)
        else:
            x_vars_temp_ = x_vars
        
        for i in range(x_vars_temp_.shape[0]):
            x_me = x_vars_temp_[i].reshape(1, -1)
            
            x_me2 = x_me.copy()
            x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 

            K = x_me.shape[1]  # Number of regressors

            if continoues_varlist[k]:
                me_variable_pr = norm.pdf(x_me @ b_pr) * b_pr[k] / 10
                me_variable_lg = (np.exp(x_me @ b_lg) / (1 + np.exp(x_me @ b_lg)) ** 2) * b_lg[k] / 10

                bb  = np.outer(b_pr,b_pr)
                xx  = np.outer(x_me, x_me)
                I_k = np.eye(K)
                gx0 = norm.pdf(x_me @ b_pr)

                grad_pr = gx0*(I_k - bb @ xx)

                gx0 = logistic.pdf(x_me @ b_lg)

                grad_lg = gx0*(I_k - bb @ xx)


            else:     
                me_variable_pr = norm.cdf(x_me2 @ b_pr) - norm.cdf(x_me @ b_pr)
                me_variable_lg = logit.G(x_me2 @ b_lg) - logit.G(x_me @ b_lg)

                gx0 = norm.pdf(x_me @ b_pr)
                gx2 = norm.pdf(x_me2 @ b_pr) 

                grad_pr = gx2*x_me2 - gx0*x_me

                gx0 = logistic.pdf(x_me @ b_lg)
                gx2 = logistic.pdf(x_me2 @ b_lg) 

                grad_lg = gx2*x_me2 - gx0*x_me
            
            me_variable_pr_list.append(me_variable_pr)
            me_variable_lg_list.append(me_variable_lg)

            grad_pr_list.append(grad_pr)
            grad_lg_list.append(grad_lg)

        grad_pr = np.mean(grad_pr_list, axis=0)
        grad_lg = np.mean(grad_lg_list, axis=0)

        def get_se(grad, cov):
            cov_me = grad @ cov @ grad.T
            return np.sqrt(np.diag(cov_me))

        se_d_pr = get_se(grad_pr, probit_results['cov'])[0]
        se_d_lg = get_se(grad_lg, logit_results['cov'])[0]

        me_variable_pr = np.mean(me_variable_pr_list)
        me_variable_lg = np.mean(me_variable_lg_list)

        results[x_lab[k]] = {
            'me_probit': me_variable_pr,
            'se_probit': se_d_pr,
            'me_logit': me_variable_lg,
            'se_logit': se_d_lg
        }

    formatted_data = []

    for variable, values in results.items():
        t_probit = values['me_probit']/values['se_probit']
        df = x_vars.shape[0] - x_vars.shape[1]
        p_val_probit = 2 * (1 - t.cdf(abs(t_probit), df=df))

        if p_val_probit < 0.01:
            stars_probit = "***"
        elif p_val_probit < 0.05:
            stars_probit = "**"
        elif p_val_probit < 0.10:
            stars_probit = "*"
        else:
            stars_probit = ""

        t_logit = values['me_logit']/values['se_logit']
        df = x_vars.shape[0] - x_vars.shape[1]
        p_val_logit = 2 * (1 - t.cdf(abs(t_logit), df=df))
        
        if p_val_logit < 0.01:
            stars_logit = "***"
        elif p_val_logit < 0.05:
            stars_logit = "**"
        elif p_val_logit < 0.10:
            stars_logit = "*"
        else:
            stars_logit = ""

        decimals = 3

        formatted_data.append([variable, f"{values['me_probit']:.{decimals}f}{stars_probit}", f"{values['me_logit']:.{decimals}f}{stars_logit}"])
        formatted_data.append(["", f"({values['se_probit']:.{decimals}f})", f"({values['se_logit']:.{decimals}f})"])

    formatted_df = pd.DataFrame(formatted_data, columns=["Variable", "Probit Model", "Logit Model"])

    return formatted_df

## Partiel Effects on Average

Note, I set 'method='BFGS''. This is not uncontroversial

In [8]:
options = {'maxiter': 10000,
           'disp': True}

method = 'BFGS'
constant = 1
average_effect = True

df_c = probit_logit_model(x_c, x_lab_c, continoues_varlist_c, options, method=method, constant=constant, average_effect=average_effect)

df_nc = probit_logit_model(x_nc, x_lab_nc, continoues_varlist_nc, options, method=method, constant=constant, average_effect=average_effect)

df_nc = df_nc.rename(columns={"Variable": "Variable NC", "Probit Model": "Probit No Controls (1)", "Logit Model": "Logit No Controls (2)"})
df_c = df_c.rename(columns={"Variable": "Variables", "Probit Model": "Probit Controls (3)", "Logit Model": "Logit Controls (4)"})

formatted_df = pd.concat([df_nc, df_c], axis=1)

formatted_df = formatted_df.drop(columns=["Variable NC"])

formatted_df = formatted_df[["Variables", "Probit No Controls (1)", "Logit No Controls (2)", "Probit Controls (3)", "Logit Controls (4)"]]

formatted_df = formatted_df.fillna("")

formatted_df

Optimization terminated successfully.
         Current function value: 0.022407
         Iterations: 80
         Function evaluations: 1344
         Gradient evaluations: 84
Optimizer succeded after 80 iter. (1344 func. evals.). Final criterion:  0.02241.
Probit, y = ['anyuseofforce_coded']
Optimization terminated successfully.
         Current function value: 0.022658
         Iterations: 113
         Function evaluations: 1824
         Gradient evaluations: 114
Optimizer succeded after 113 iter. (1824 func. evals.). Final criterion:  0.02266.
Logit, y = ['anyuseofforce_coded']
Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 37
         Function evaluations: 195
         Gradient evaluations: 39
Optimizer succeded after 37 iter. (195 func. evals.). Final criterion:  0.03044.
Probit, y = ['anyuseofforce_coded']
Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 51
         Function ev

,Variables,Probit No Controls (1),Logit No Controls (2),Probit Controls (3),Logit Controls (4)
0,sblack,0.004,0.004,0.001,0.000
1,,(0.005),(0.005),(0.002),(0.001)
2,shisp,0.013*,0.013*,0.002,0.001
3,,(0.007),(0.007),(0.003),(0.002)
4,sother,0.003,0.003,0.000,-0.000
5,,(0.007),(0.007),(0.002),(0.002)
6,smale,,,0.001,0.001
7,,,,(0.001),(0.001)
8,sempl,,,-0.000,-0.000
9,,,,(0.000),(0.000)


#### Construct a pretty latex table

In [9]:
df_nc = df_nc.rename(columns={"Variable": "Variable NC", "Probit Model": "Probit No Controls (1)", "Logit Model": "Logit No Controls (2)"})
df_c = df_c.rename(columns={"Variable": "Variables", "Probit Model": "Probit Controls (3)", "Logit Model": "Logit Controls (4)"})

formatted_df = pd.concat([df_nc, df_c], axis=1)

formatted_df = formatted_df.drop(columns=["Variable NC"])

formatted_df = formatted_df[["Variables", "Probit No Controls (1)", "Logit No Controls (2)", "Probit Controls (3)", "Logit Controls (4)"]]

formatted_df = formatted_df.fillna("")

formatted_df = formatted_df.rename(columns={"Probit No Controls (1)": "Probit NC", "Logit No Controls (2)": "Logit NC", "Probit Controls (3)": "Probit C", "Logit Controls (4)": "Logit C"})

latex_table = formatted_df.to_latex(index=False, header=True, column_format="lcccc", escape=False)

latex_note = "\n\\begin{tablenotes}\n\\small \n\\item \\textbf{Note:} Standard errors in parenthesis. \\(* p < 0.10, ** p < 0.05, *** p < 0.01\\). \n\\end{tablenotes}"

# Combine table and note
complete_latex = "\\begin{table}[ht]\n\\centering\n\\caption{Partial effects at the average}\n" + latex_table + latex_note + "\n\\end{table}"

print(complete_latex)

\begin{table}[ht]
\centering
\caption{Partial effects at the average}
\begin{tabular}{lcccc}
\toprule
Variables & Probit NC & Logit NC & Probit C & Logit C \\
\midrule
sblack & 0.004 & 0.004 & 0.001 & 0.000 \\
 & (0.005) & (0.005) & (0.002) & (0.001) \\
shisp & 0.013* & 0.013* & 0.002 & 0.001 \\
 & (0.007) & (0.007) & (0.003) & (0.002) \\
sother & 0.003 & 0.003 & 0.000 & -0.000 \\
 & (0.007) & (0.007) & (0.002) & (0.002) \\
smale &  &  & 0.001 & 0.001 \\
 &  &  & (0.001) & (0.001) \\
sempl &  &  & -0.000 & -0.000 \\
 &  &  & (0.000) & (0.000) \\
sincome_medium &  &  & 0.000 & 0.000 \\
 &  &  & (0.001) & (0.001) \\
sincome_high &  &  & 0.000 & 0.000 \\
 &  &  & (0.001) & (0.001) \\
spop_medium &  &  & 0.003 & 0.003 \\
 &  &  & (0.002) & (0.002) \\
spop_large &  &  & 0.002 & 0.003 \\
 &  &  & (0.007) & (0.007) \\
spop_huge &  &  & 0.002 & 0.003 \\
 &  &  & (0.005) & (0.005) \\
inctype_traffic &  &  & -0.000 & -0.000 \\
 &  &  & (0.000) & (0.000) \\
daytime &  &  & -0.000 & -0.000 \\
 &  

## Average Partial Effects

In [10]:
options = {'maxiter': 10000,
           'disp': True}

method = 'BFGS'
constant = 1
average_effect = False

df_c = probit_logit_model(x_c, x_lab_c, continoues_varlist_c, options, method=method, constant=constant, average_effect=average_effect)

df_nc = probit_logit_model(x_nc, x_lab_nc, continoues_varlist_nc, options, method=method, constant=constant, average_effect=average_effect)


df_nc = df_nc.rename(columns={"Variable": "Variable NC", "Probit Model": "Probit No Controls (1)", "Logit Model": "Logit No Controls (2)"})
df_c = df_c.rename(columns={"Variable": "Variables", "Probit Model": "Probit Controls (3)", "Logit Model": "Logit Controls (4)"})

formatted_df = pd.concat([df_nc, df_c], axis=1)

formatted_df = formatted_df.drop(columns=["Variable NC"])

formatted_df = formatted_df[["Variables", "Probit No Controls (1)", "Logit No Controls (2)", "Probit Controls (3)", "Logit Controls (4)"]]

formatted_df = formatted_df.fillna("")

formatted_df = formatted_df.rename(columns={"Probit No Controls (1)": "Probit NC", "Logit No Controls (2)": "Logit NC", "Probit Controls (3)": "Probit C", "Logit Controls (4)": "Logit C"})

formatted_df

Optimization terminated successfully.
         Current function value: 0.022407
         Iterations: 80
         Function evaluations: 1344
         Gradient evaluations: 84
Optimizer succeded after 80 iter. (1344 func. evals.). Final criterion:  0.02241.
Probit, y = ['anyuseofforce_coded']
Optimization terminated successfully.
         Current function value: 0.022658
         Iterations: 113
         Function evaluations: 1824
         Gradient evaluations: 114
Optimizer succeded after 113 iter. (1824 func. evals.). Final criterion:  0.02266.
Logit, y = ['anyuseofforce_coded']
Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 37
         Function evaluations: 195
         Gradient evaluations: 39
Optimizer succeded after 37 iter. (195 func. evals.). Final criterion:  0.03044.
Probit, y = ['anyuseofforce_coded']
Optimization terminated successfully.
         Current function value: 0.030440
         Iterations: 51
         Function ev

,Variables,Probit NC,Logit NC,Probit C,Logit C
0,sblack,0.005,0.005,0.002,0.001
1,,(0.005),(0.006),(0.005),(0.004)
2,shisp,0.013*,0.013*,0.005,0.004
3,,(0.007),(0.007),(0.005),(0.004)
4,sother,0.003,0.003,0.001,-0.001
5,,(0.007),(0.008),(0.008),(0.006)
6,smale,,,0.002**,0.002*
7,,,,(0.001),(0.001)
8,sempl,,,-0.002,-0.002
9,,,,(0.001),(0.001)


In [11]:
df_nc = df_nc.rename(columns={"Variable": "Variable NC", "Probit Model": "Probit No Controls (1)", "Logit Model": "Logit No Controls (2)"})
df_c = df_c.rename(columns={"Variable": "Variables", "Probit Model": "Probit Controls (3)", "Logit Model": "Logit Controls (4)"})

formatted_df = pd.concat([df_nc, df_c], axis=1)

formatted_df = formatted_df.drop(columns=["Variable NC"])

formatted_df = formatted_df[["Variables", "Probit No Controls (1)", "Logit No Controls (2)", "Probit Controls (3)", "Logit Controls (4)"]]

formatted_df = formatted_df.rename(columns={"Probit No Controls (1)": "Probit NC", "Logit No Controls (2)": "Logit NC", "Probit Controls (3)": "Probit C", "Logit Controls (4)": "Logit C"})

formatted_df = formatted_df.fillna("")

latex_table = formatted_df.to_latex(index=False, header=True, column_format="lcccc", escape=False)

latex_note = "\n\\begin{tablenotes}\n\\small \n\\item \\textbf{Note:} Standard errors in parenthesis. \\(* p < 0.10, ** p < 0.05, *** p < 0.01\\). \n\\end{tablenotes}"

# Combine table and note
complete_latex = "\\begin{table}[ht]\n\\centering\n\\caption{Average partial effects}\n" + latex_table + latex_note + "\n\\end{table}"

print(complete_latex)

\begin{table}[ht]
\centering
\caption{Average partial effects}
\begin{tabular}{lcccc}
\toprule
Variables & Probit NC & Logit NC & Probit C & Logit C \\
\midrule
sblack & 0.005 & 0.005 & 0.002 & 0.001 \\
 & (0.005) & (0.006) & (0.005) & (0.004) \\
shisp & 0.013* & 0.013* & 0.005 & 0.004 \\
 & (0.007) & (0.007) & (0.005) & (0.004) \\
sother & 0.003 & 0.003 & 0.001 & -0.001 \\
 & (0.007) & (0.008) & (0.008) & (0.006) \\
smale &  &  & 0.002** & 0.002* \\
 &  &  & (0.001) & (0.001) \\
sempl &  &  & -0.002 & -0.002 \\
 &  &  & (0.001) & (0.001) \\
sincome_medium &  &  & 0.001 & 0.001 \\
 &  &  & (0.004) & (0.003) \\
sincome_high &  &  & 0.002 & 0.001 \\
 &  &  & (0.003) & (0.003) \\
spop_medium &  &  & 0.007* & 0.007* \\
 &  &  & (0.004) & (0.004) \\
spop_large &  &  & 0.007 & 0.009 \\
 &  &  & (0.016) & (0.018) \\
spop_huge &  &  & 0.007 & 0.009 \\
 &  &  & (0.011) & (0.012) \\
inctype_traffic &  &  & -0.001 & -0.001 \\
 &  &  & (0.001) & (0.001) \\
daytime &  &  & -0.001 & -0.001 \\
 &  & 

## OLD

In [12]:
# def probit_logit_model(x_vars, x_lab, continoues_varlist, options, method='BFGS', constant=1, average_effect=True):

#     theta0 = probit.starting_values(y, x_vars)
#     probit_results = est.estimate(probit.q, theta0, y, x_vars, options=options, method=method)

#     probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')

#     theta0 = logit.starting_values(y, x_vars)
#     logit_results = est.estimate(logit.q, theta0, y, x_vars, options=options, method=method)
#     logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')

#     b_pr = probit_tab.theta.values
#     b_lg = logit_tab.theta.values

#     results = {}

#     for k in range(x_vars.shape[1]-constant): 

#         me_variable_pr_list = []
#         se_d_pr_list = []
#         me_variable_lg_list = []
#         se_d_lg_list = []

#         if average_effect:
#             x_vars_temp_ = np.mean(x_vars, axis=0).reshape(1, -1)
#         else:
#             x_vars_temp_ = x_vars
        
#         for i in range(x_vars_temp_.shape[0]):
#             x_me = x_vars_temp_[i].reshape(1, -1)
            
#             x_me2 = x_me.copy()
#             x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 

#             K = x_me.shape[1]  # Number of regressors

#             if continoues_varlist[k]:
#                 me_variable_pr = norm.pdf(x_me @ b_pr) * b_pr[k] / 10
#                 me_variable_lg = (np.exp(x_me @ b_lg) / (1 + np.exp(x_me @ b_lg)) ** 2) * b_lg[k] / 10

#                 bb  = np.outer(b_pr,b_pr)
#                 xx  = np.outer(x_me, x_me)
#                 I_k = np.eye(K)
#                 gx0 = norm.pdf(x_me @ b_pr)

#                 grad_pr = gx0*(I_k - bb @ xx)

#                 gx0 = logistic.pdf(x_me @ b_lg)

#                 grad_lg = gx0*(I_k - bb @ xx)


#             else:     
#                 me_variable_pr = norm.cdf(x_me2 @ b_pr) - norm.cdf(x_me @ b_pr)
#                 me_variable_lg = logit.G(x_me2 @ b_lg) - logit.G(x_me @ b_lg)

#                 gx0 = norm.pdf(x_me @ b_pr)
#                 gx2 = norm.pdf(x_me2 @ b_pr) 

#                 grad_pr = gx2*x_me2 - gx0*x_me

#                 gx0 = logistic.pdf(x_me @ b_lg)
#                 gx2 = logistic.pdf(x_me2 @ b_lg) 

#                 grad_lg = gx2*x_me2 - gx0*x_me
            


#             def get_se(grad, cov):
#                 cov_me = grad @ cov @ grad.T
#                 return np.sqrt(np.diag(cov_me))

#             se_d_pr = get_se(grad_pr, probit_results['cov'])
#             se_d_lg = get_se(grad_lg, logit_results['cov'])

#             me_variable_pr_list.append(me_variable_pr)
#             se_d_pr_list.append(se_d_pr)
#             me_variable_lg_list.append(me_variable_lg)
#             se_d_lg_list.append(se_d_lg)

#         me_variable_pr = np.mean(me_variable_pr_list)
#         se_d_pr = np.mean(se_d_pr_list)
#         me_variable_lg = np.mean(me_variable_lg_list)
#         se_d_lg = np.mean(se_d_lg_list)

#         results[x_lab[k]] = {
#             'me_probit': me_variable_pr,
#             'se_probit': se_d_pr,
#             'me_logit': me_variable_lg,
#             'se_logit': se_d_lg
#         }

#     formatted_data = []

#     for variable, values in results.items():
#         t_probit = values['me_probit']/values['se_probit']
#         df = x_vars.shape[0] - x_vars.shape[1]
#         p_val_probit = 2 * (1 - t.cdf(abs(t_probit), df=df))

#         if p_val_probit < 0.01:
#             stars_probit = "***"
#         elif p_val_probit < 0.05:
#             stars_probit = "**"
#         elif p_val_probit < 0.10:
#             stars_probit = "*"
#         else:
#             stars_probit = ""

#         t_logit = values['me_logit']/values['se_logit']
#         df = x_vars.shape[0] - x_vars.shape[1]
#         p_val_logit = 2 * (1 - t.cdf(abs(t_logit), df=df))
        
#         if p_val_logit < 0.01:
#             stars_logit = "***"
#         elif p_val_logit < 0.05:
#             stars_logit = "**"
#         elif p_val_logit < 0.10:
#             stars_logit = "*"
#         else:
#             stars_logit = ""

#         decimals = 5

#         formatted_data.append([variable, f"{values['me_probit']:.{decimals}f}{stars_probit}", f"{values['me_logit']:.{decimals}f}{stars_logit}"])
#         formatted_data.append(["", f"({values['se_probit']:.{decimals}f})", f"({values['se_logit']:.{decimals}f})"])

#     formatted_df = pd.DataFrame(formatted_data, columns=["Variable", "Probit Model", "Logit Model"])

#     return formatted_df, se_d_pr_list